In [1]:
%pip install osmium folium memory_profiler

Note: you may need to restart the kernel to use updated packages.


In [2]:
#aco_t1 = time.time()
#aco = sa.ACO(cities = ciudades3, start_city = start_city)
#aco_solution, aco_cost = aco.solve()
#aco_t2 = time.time()

In [3]:
import vrp
import geographs

In [4]:
Depot_coord = (18.463521, -69.934688)#(18.481891, -69.913954)8299470296
qty_clients = 30
cap_trucks = 80

In [5]:
import random
inv = []
qty_trucks = 3
for j in range(qty_clients):
    inv.append(random.randint(0, qty_trucks-1))

print(inv)

[2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 2, 1]


In [6]:
print("Loading data... ")

geoGraph = geographs.GeoGraph(limit_coords=Depot_coord)
print("Data loaded! ")

Loading data... 
Available nodes before condition: 133161
Available nodes after condition: 19752
Data loaded! 


In [7]:
Depot_node, clients, qty_trucks = vrp.create_nodes(geoGraph, Depot_coord, qty_clients, cap_trucks)

print("Depot node: ", Depot_node)
print("qty of trucks: ", qty_trucks)

Depot node:  GeoNode 940894323 at pos (-699350746, 184632713), coords (18.4632713, -69.9350746)
qty of trucks:  3


In [8]:
qty_poblacion = 200
n_elite = 10
n_generations = 50
prob_de_mut = 0.5

create_clusters = vrp.create_clusters(geoGraph, qty_clients, qty_trucks, cap_trucks, qty_poblacion, n_elite, n_generations, clients, prob_de_mut)
solution = create_clusters.create_clusters()
print("solution: ", solution)

solution:  [2, 0, 1, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 2, 0, 2, 0, 2, 0, 2, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0]


In [9]:
n_nearest_nodes = 3

routes, paths = vrp.create_routes(geoGraph, Depot_node, clients, qty_trucks, solution, n_nearest_nodes)

print("routes: ", routes)

for graph in routes:
    print(graph.get_nodes())


Truck 1/3
Connecting 310836814 and 310832291
Connecting 310836814 and 310762931
Connecting 310836814 and 309575502
Connecting 310836814 and 940894323
Connecting 6691385785 and 309575502
Connecting 6691385785 and 310762931
Connecting 6691385785 and 310836814
Connecting 6691385785 and 940894323
Connecting 6302357546 and 6689702627
Connecting 6302357546 and 309558923
Connecting 6302357546 and 310762931
Connecting 6302357546 and 940894323
Connecting 310762931 and 310836814
Connecting 310762931 and 310832291
Connecting 310762931 and 6691385785
Connecting 310762931 and 940894323
Connecting 309575502 and 6691385785
Connecting 309575502 and 310762931
Connecting 309575502 and 310836814
Connecting 309575502 and 940894323
Connecting 310832291 and 310836814
Connecting 310832291 and 310762931
Connecting 310832291 and 309575502
Connecting 310832291 and 940894323
Connecting 6689702627 and 6302357546
Connecting 6689702627 and 309558923
Connecting 6689702627 and 310762931
Connecting 6689702627 and 9408

In [10]:
print(geoGraph.get_node_by_name(309802646))

GeoNode 309802646 at pos (-699188347, 185186788), coords (18.5186788, -69.9188347)


In [11]:
print(geoGraph.get_node_by_name(310233110))

GeoNode 310233110 at pos (-699169869, 185010114), coords (18.5010114, -69.9169869)


In [12]:
Depot_node.name

940894323

In [13]:
import folium
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route in paths:
    for initial_node_name in route:
        for goal_node_name in route[initial_node_name]:
            
            initial_node = routes[i].get_node_by_name(initial_node_name)
            goal_node = routes[i].get_node_by_name(goal_node_name)

            initial_marker = [initial_node.lat, initial_node.lon]
            folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
            
            goal_marker = [goal_node.lat, goal_node.lon]
            folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

            path = route[initial_node_name][goal_node_name]
            points = []

            for node in path:
                point = [node.lat, node.lon]
                points.append(point)

            folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [14]:
import genetic_algorithms_vin as gav
import random

answers = []

for route in routes:

    SUCCESS = False

    while not SUCCESS:

        graph = route.get_adj_matrix(Depot_node)

        #Params
        num_gen = 5000
        population_num = 200
        solutions_to_choose = 5
        mutation_prob = 0.9

        cities = list(range(0,len(graph)))

        initial_city = 0

        #initial population
        solutions = gav.create_population(population_num, cities, initial_city)
        #print('Propulation: ', solutions)

        best_solutions_by_gen = []

        #For each generation
        for gen in range(num_gen):

            #Calculate the total distance of each solution
            distances = gav.calc_distances_paths(graph, initial_city, solutions)
            #print("distances: ", distances)
            
            #Calculate the fitness for each solution
            fitnesess = gav.calc_fitness(distances)
            #print("fitnesess: ", fitnesess)

            if(sum(fitnesess) == 0):
                choosen_solutions = solutions[:solutions_to_choose]
            else:
                #Choose k solutions randomly based on their fitnesess
                choosen_solutions = random.choices(solutions, weights=fitnesess, k=solutions_to_choose)
            #print("choosen solutions", choosen_solutions)

            #Save the best k solutions of this generation
            best_solutions_by_gen.append(choosen_solutions)

            #Use best solutions to create a new generation by croosover and mutation (based in mutation_prob)
            crossovered_solutions = gav.crossover_solutions(choosen_solutions, population_num)
            #print("Crossovered_solutions: ", crossovered_solutions)
            solutions = gav.mutate_solutions(crossovered_solutions, mutation_prob)
            #print("Mutated_solutions: ", solutions)

        best_solution_by_gen = []

        for gen_solutions in best_solutions_by_gen:
            distances = gav.calc_distances_paths(graph, initial_city, gen_solutions)
            best_solution_of_gen = gav.sort_solutions_by_distance(gen_solutions, distances)[-1]
            best_solution_by_gen.append(best_solution_of_gen)

        final_answer, cost = sorted(best_solution_by_gen,key=lambda x: x[1])[0]

        final_answer.append(initial_city)

        print("Answer: ", final_answer, cost)

        SUCCESS = cost != float('inf')

    answers.append(final_answer)

Answer:  [0, 10, 7, 4, 2, 5, 3, 1, 6, 9, 8, 0] inf
Answer:  [0, 10, 1, 5, 3, 7, 2, 4, 9, 8, 6, 0] inf
Answer:  [0, 7, 4, 2, 10, 5, 6, 1, 8, 9, 3, 0] inf
Answer:  [0, 10, 3, 8, 1, 5, 7, 6, 4, 9, 2, 0] inf
Answer:  [0, 9, 10, 7, 8, 1, 4, 6, 2, 5, 3, 0] inf
Answer:  [0, 9, 7, 10, 2, 1, 3, 5, 8, 6, 4, 0] inf
Answer:  [0, 8, 7, 3, 4, 1, 6, 2, 9, 10, 5, 0] inf
Answer:  [0, 9, 4, 10, 7, 2, 1, 5, 6, 3, 8, 0] inf
Answer:  [0, 8, 6, 7, 1, 5, 9, 3, 10, 2, 4, 0] inf
Answer:  [0, 10, 6, 3, 2, 1, 4, 5, 9, 8, 7, 0] inf
Answer:  [0, 8, 1, 4, 10, 7, 6, 3, 5, 2, 9, 0] inf
Answer:  [0, 10, 7, 2, 6, 4, 9, 1, 3, 8, 5, 0] inf
Answer:  [0, 9, 2, 10, 4, 5, 3, 6, 1, 7, 8, 0] inf
Answer:  [0, 10, 2, 9, 4, 3, 5, 6, 1, 7, 8, 0] inf
Answer:  [0, 10, 4, 9, 1, 3, 8, 6, 2, 7, 5, 0] inf
Answer:  [0, 6, 9, 2, 7, 1, 5, 8, 3, 4, 10, 0] inf
Answer:  [0, 10, 4, 2, 3, 9, 7, 5, 1, 8, 6, 0] inf
Answer:  [0, 9, 8, 6, 1, 3, 7, 10, 5, 2, 4, 0] inf
Answer:  [0, 10, 9, 3, 5, 2, 7, 4, 8, 1, 6, 0] inf
Answer:  [0, 9, 4, 1, 8, 3, 6, 

KeyboardInterrupt: 

In [ ]:
import folium
colors = ['red', 'blue', 'green', 'purple', 'darkblue']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route_graph,route, answer in zip(routes,paths, answers):
    
    nodes_answers = [route_graph.index_to_node[answer_node] for answer_node in answer]

    print("Nodes answers: ", nodes_answers)

    j = 0
    for j in range(len(nodes_answers)-1):
        initial_node = routes[i].get_node_by_name(nodes_answers[j])
        goal_node = routes[i].get_node_by_name(nodes_answers[j+1])

        initial_marker = [initial_node.lat, initial_node.lon]
        folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
        
        goal_marker = [goal_node.lat, goal_node.lon]
        folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

        print("Initial node: ", initial_node)
        print("goal node: ", goal_node)

        path = route[initial_node.name][goal_node.name]
        points = []

        for node in path:
            point = [node.lat, node.lon]
            points.append(point)

        folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

Nodes answers:  [4100826966, 309565654, 4117721277, 9244572806, 308689423, 308908454, 308923738, 1860279191, 1860751308, 4100826966]
Initial node:  GeoNode 4100826966 at pos (-699137564, 184818858), coords (18.4818858, -69.9137564)
goal node:  GeoNode 309565654 at pos (-699279161, 184432439), coords (18.4432439, -69.9279161)
Initial node:  GeoNode 309565654 at pos (-699279161, 184432439), coords (18.4432439, -69.9279161)
goal node:  GeoNode 4117721277 at pos (-699222001, 184508725), coords (18.4508725, -69.9222001)
Initial node:  GeoNode 4117721277 at pos (-699222001, 184508725), coords (18.4508725, -69.9222001)
goal node:  GeoNode 9244572806 at pos (-699035123, 184604569), coords (18.4604569, -69.9035123)
Initial node:  GeoNode 9244572806 at pos (-699035123, 184604569), coords (18.4604569, -69.9035123)
goal node:  GeoNode 308689423 at pos (-698992953, 184752395), coords (18.4752395, -69.8992953)
Initial node:  GeoNode 308689423 at pos (-698992953, 184752395), coords (18.4752395, -69.8